# Test EV control

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from ev_ssm import ev_ssm
from tqdm import tqdm

In [2]:
%run -i 'cosim_import.py'

0.post4466.dev0+g553664b
2.8.0


In [3]:
caseH = 18
%run -i 'cosim_setup_andes.py'

EVA: ts=18[H], 50000 EVs, Total Q=434.64 MWh
Online 14481, Q=361.83 MWh, SoC=0.7687
Power(MW): Pt=-39.9433, Pc=-39.9433, Pd=-0.0
Ctrl: 0=7778; 1=6703; 
EVA: Load A from Aest.csv.
Working directory: "/Users/jinningwang/Documents/work/andes/jwang/notes"
> Loaded config from file "/Users/jinningwang/.andes/andes.rc"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Generated code for <PQ> is stale.
Numerical code generation (rapid incremental mode) started...


Generating code for 1 models on 8 processes.


Saved generated pycode to "/Users/jinningwang/.andes/pycode"
> Reloaded generated Python code of module "pycode".
Generated numerical code for 1 models in 0.1620 seconds.
Parsing input file "/Users/jinningwang/Documents/work/andes/jwang/case/ieee39_ev2.xlsx"...
Input file parsed in 0.0702 seconds.
System internal structure set up in 0.0237 seconds.


In [4]:
%run -i 'cosim_loadsyn.py'

ANDES total load is: 58.564 p.u.


In [5]:
%run -i 'cosim_setup_dispatch.py'

-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: On
   Sparse solver: KLU
 Solution method: NR method
Numba compilation initiated with caching.
Power flow initialized in 0.1166 seconds.
0: |F(x)| = 7.561234474
1: |F(x)| = 2.917099456
2: |F(x)| = 0.1313037751
3: |F(x)| = 0.0003572374622
4: |F(x)| = 4.286882849e-09
Converged in 5 iterations in 0.0066 seconds.
Power flow results are consistent. Conversion is successful.


Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-08


-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: On
   Sparse solver: KLU
 Solution method: NR method
Numba compilation initiated with caching.
Power flow initialized in 0.0039 seconds.
0: |F(x)| = 7.561234474
1: |F(x)| = 2.917099456
2: |F(x)| = 0.1313037751
3: |F(x)| = 0.0003572374622
4: |F(x)| = 4.286882849e-09
Converged in 5 iterations in 0.0033 seconds.
Power flow results are consistent. Conversion is successful.
GSF is consistent.
RTED GB model is loaded.
RTED removed: ['rampu', 'rampd']
RTED limited: sfru, sfrd
RTED: total cost=4910.307


pp cost=2224.8754288341015, gb cost=4910.307428177316


In [6]:
%run -i 'cosim_linktable.py'

In [7]:
# Change ``rru``, ``rrd``, ``rsfr``, ``t_total`` for necessary.
t_total = 3600

rru, rrd, rsfr = 1.0, 1.0, 0.04

# Define functions and constants used in loop
%run -i 'cosim_const.py'

Analyze the data to initialize the number of actions.

In [28]:
# read data from previous run
evsr1 = pd.read_csv('cosim_18_F_evsoc1.csv')
evsr2 = pd.read_csv('cosim_18_F_evsoc2.csv')
evd = pd.read_csv('cosim_18_F_evdata.csv').iloc[ridx].reset_index(drop=True)
eva = pd.read_csv('cosim_18_F_evagc.csv')

evs = pd.concat([evsr1, evsr2], ignore_index=True)

In [29]:
# reconstruct the EV Ctrl signal
evc = pd.DataFrame(-2*np.ones((1600, t_total)), columns=range(t_total))
evc[0] = evd['c']

for i in range(1, t_total):
    evc[i] = evc[i-1] + eva[str(i)]

In [55]:
# build the EV NA
evna = pd.DataFrame(np.zeros((1600, t_total)), columns=range(t_total))
for i in range(t_total):
    evna[i] = evs[str(i)] <= 0.96 # this time using 0.96 rather than 'socd'
    evna[i] = evna[i].astype(int)
    evna[i] = evna[i] - evc[i]

In [58]:
evna.sum(axis=1).idxmin()

813

In [54]:
evc.iloc[1530:1535]

,0,1,2,3,4,5,6,7,8,9,...,3590,3591,3592,3593,3594,3595,3596,3597,3598,3599
1530,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1531,0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1532,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1533,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1534,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
evna.iloc[1530:1535]

,0,1,2,3,4,5,6,7,8,9,...,3590,3591,3592,3593,3594,3595,3596,3597,3598,3599
1530,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1531,1,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1532,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1533,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1534,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
evs.iloc[1530:1535]

,0,1,2,3,4,5,6,7,8,9,...,3590,3591,3592,3593,3594,3595,3596,3597,3598,3599
1530,0.243808,0.243808,0.243808,0.243808,0.243808,0.243808,0.243808,0.243808,0.243808,0.243808,...,0.256825,0.256881,0.256937,0.256993,0.257049,0.257105,0.257161,0.257217,0.257273,0.257330
1531,0.784452,0.784497,0.784497,0.784542,0.784542,0.784542,0.784542,0.784542,0.784586,0.784631,...,0.819092,0.819092,0.819092,0.819092,0.819092,0.819092,0.819092,0.819092,0.819092,0.819092
1532,0.239836,0.239836,0.239836,0.239836,0.239836,0.239836,0.239836,0.239836,0.239836,0.239836,...,0.347691,0.347753,0.347816,0.347879,0.347941,0.348004,0.348066,0.348129,0.348191,0.348254
1533,0.248131,0.248131,0.248131,0.248131,0.248131,0.248131,0.248131,0.248131,0.248131,0.248131,...,0.439530,0.439596,0.439662,0.439729,0.439795,0.439862,0.439928,0.439994,0.440061,0.440127
1534,0.886985,0.886985,0.886985,0.886985,0.886985,0.886985,0.886985,0.886985,0.886985,0.886985,...,0.889542,0.889542,0.889542,0.889542,0.889542,0.889542,0.889542,0.889542,0.889542,0.889542


In [53]:
evd[['soci', 'socd']].iloc[1530:1535]

,soci,socd
1530,0.243808,0.775142
1531,0.246902,0.785703
1532,0.239836,0.831588
1533,0.248131,0.857322
1534,0.330185,0.820864


In [44]:
evna

,0,1,2,3,4,5,6,7,8,9,...,3590,3591,3592,3593,3594,3595,3596,3597,3598,3599
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1596,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1597,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1598,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Revise the EV control logic as: C->I, I->D, SOC high to low

In [ ]:
# --- Response ---

sse = ev_ssm(ts=18, N=10000, step=1, tp=40,
             lr=0.1, lp=40, seed=2022, name="EVA")
sse.load_A("Aest.csv")


In [ ]:
sse.ev['Qa'] = (sse.ev['tf'] - sse.ev['ts']) * sse.ev['Pc'] * sse.ev['nc'] - sse.ev['socd'] * sse.ev['Q']
sse.ev['Qr'] = sse.ev['Qa'].mean() + sse.ev['tt'] * sse.ev['Pd']
sse.ev['nr'] = sse.ev['Qr'] / sse.ev['Pc'] / sse.ev['nc'] * 3600 / 4

sse.ev['nr'].hist()

sse.ev['Qa'].mean()

In [ ]:
# --- Response ---

sse = ev_ssm(ts=10.4, N=10000, step=1, tp=40,
             lr=0.1, lp=40, seed=2022, name="EVA")
sse.load_A("Aest.csv")

x0 = sse.x0.copy()
print(sse.Prc)
sse.run(tf=10.4 + 2/3600, Pi=9,
        is_updateA=False, is_rstate=True,
        is_test=False, disable=False)
print(sse.Prc)
x1 = sse.x0.copy()

In [ ]:
plt.bar(range(len(x0)), x0, label="x0")
plt.bar(range(len(x0)), x1, label="x0", alpha=0.5)

In [ ]:
sse = ev_ssm(ts=10.4, N=10000, step=1, tp=40,
             lr=0.1, lp=40, seed=2022, name="EVA",
             is_report=False)

# historical data
ev_num = pd.read_csv("ev_num.csv")

# calculate the sf
k1 = ev_num['ne'][(ev_num['time'] >= sse.ts) & (ev_num['time'] <= sse.ts+1/12)].mean()
k0 = ev_num['ne'][ev_num['time'] >= sse.ts].iloc[0]
k = k1 / k0

# estiamte FRC
sse.g_frc(nea=sse.ne*k)

In [ ]:
## Run to get the EV numbers within a day

# import joblib
# from joblib import Parallel, delayed

# Nt = 10000
# def rne(time):
#     sse1 = ev_ssm(ts=time, N=Nt, step=1, tp=40,
#                   lr=0.1, lp=60, seed=2022, name="EVA",
#                   is_report=False)
#     return sse1.ne

# # interval by minutes, 24H
# rnel = Parallel(n_jobs=joblib.cpu_count())(delayed(rne)(i) for i in np.arange(1/60, 24, 1/60))

# ev_num = pd.DataFrame()
# ev_num['time'] = np.arange(1/60, 24, 1/60)
# ev_num['ne'] = rnel
# ev_num.to_csv("ev_num.csv", index=False)